This notebook is to check whether or not Track the Recovery Data includes the requirements below. I'll be bringing in unfiltered datasets and filtering them and exporting them into their own csv files.  

__Data requirements__:
- Data includes:
    (1) SD Data
    (2) CA Data
    (3) US Country Level Data
    
__Link to datasets__: [link](https://github.com/OpportunityInsights/EconomicTracker/tree/main/data)
- DON'T include the Google Mobility Data or the data that isn't updated regularly


In [1]:
# try to automate the extraction from Github folder to the raw_data folder of Track the Recovery Folder
# can we start downloading these files onto the M drive in order to transfer these in the future?

In [2]:
# IMPORT STATEMENTS

import os
import pandas as pd
import datetime
from pandas._testing import assert_frame_equal

In [86]:
def sd_city(filename):
    '''Filters Economic Tracker city level data to only include San Diego County'''
    sd_fips_code = 6
    path = os.path.join('raw_data', filename)
    city = pd.read_csv(path)
    city = city[city['cityid'] == sd_fips_code]
    city = clean_df(city)
    if last_week_is_same(city, filename):
        city.to_csv('temp\{}'.format(filename), index = False)
        name_files(city, filename)
    else:
        raise ValueError('Last week\'s data not in the same format as this week\'s\n')

def sd_county(filename):
    '''Filters Economic Tracker county level data to only include San Diego County'''
    sd_fips_code = 6073
    path = os.path.join('raw_data', filename)
    county = pd.read_csv(path)
    county = county[county['countyfips'] == sd_fips_code]
    #not all csv files have unsorted dates, but this is in case they ever do 
    county = clean_df(county)
    if last_week_is_same(county, filename):
        county.to_csv('temp\{}'.format(filename), index = False)
        name_files(county, filename)
    else:
        raise ValueError('Last week\'s data not in the same format as this week\'s\n')
    
def state_ca(filename):
    '''Filters Economic Tracker state level data to only include California'''
    ca_fips_code = 6
    path = os.path.join('raw_data', filename)
    state = pd.read_csv(path)
    state = state[state['statefips'] == ca_fips_code]
    state = clean_df(state)
    if last_week_is_same(state, filename):
        state.to_csv('temp\{}'.format(filename), index = False)
        name_files(state, filename)
    else:
        raise ValueError('Last week\'s data not in the same format as this week\'s\n')

def us_national(filename):
    '''Moves Economic Tracker national level data to filtered_data folder'''
    path = os.path.join('raw_data', filename)
    us = pd.read_csv(path)
    us = clean_df(us)
    if last_week_is_same(us, filename):
        us.to_csv('temp\{}'.format(filename), index = False)
        name_files(us, filename)
    else:
        raise ValueError('Last week\'s data not in the same format as this week\'s\n')
            

In [87]:
def clean_df(df):
    if 'initial_claims' in df.columns:
        df['initial_claims'] = df['initial_claims'].astype(int)
    if'total_claims' in df.columns:
        df['total_claims'] = df['total_claims'].astype(int)
    if 'engagement' in df.columns:
        for i in df.columns:
            if df[i].dtype == object:
                df[i] = df[i].astype(float)
    if 'day_endofweek' in df.columns:
        df = df.rename(columns={"day_endofweek": "day"})
    df['date'] = pd.to_datetime(df[['year','month','day']])
    df = df.sort_values(by=['date'])
    if 'Daily' in filename and 'Womply' in filename:
        df = womply_daily_to_weekly(df)
    if 'Daily' in filename and 'Womply' not in filename:
        if 'Affinity' in filename:
            # the values before this date on the raw dataframes were blank
            df = df[df['date'] >= '2020-01-13 00:00:00']
        df = affinity_daily_to_weekly(df)
    if 'day_endofweek' in df.columns:
        df = df.rename(columns={"day": "day_endofweek"})
    df = df.drop(['date'], axis = 1)
    return df

In [88]:
def affinity_daily_to_weekly(df):
    '''Converts daily data to weekly data from Affinity to match SafeGraph weeks'''
    if 'cityid' in df.columns and 'spend_all' in df.columns:
        agg_dict = {'year': 'last',
                    'month': 'last',
                    'day': 'last',
                    'cityid': 'last',
                    'spend_acf': 'last',
                    'spend_aer' : 'last',
                    'spend_all' : 'last',
                    'spend_apg' : 'last',
                    'spend_grf' : 'last',
                    'spend_hcs': 'last',
                    'spend_tws' : 'last',
                    'date' : 'last'
                   }
        df = df.resample('W-Mon', on = 'date').agg(agg_dict)
        df = df[df['date'] <= ('2020-08-03 00:00:00')].reset_index(drop=True) #UPDATE
        return df
    elif 'countyfips' in df.columns and 'spend_all' in df.columns:
        agg_dict = {'year': 'last',
                    'month': 'last',
                    'day': 'last',
                    'countyfips': 'last',
                    'spend_all': 'last',
                    'date': 'last'}
        df = df.resample('W-Mon', on = 'date').agg(agg_dict)
        df = df[df['date'] <= ('2020-08-03 00:00:00')].reset_index(drop=True) #UPDATE
        return df
    elif 'statefips' in df.columns and 'spend_all' in df.columns:
        agg_dict = {'year': 'last',
                    'month': 'last',
                    'day': 'last',
                    'statefips': 'last',
                    'spend_acf': 'last',
                    'spend_aer' : 'last',
                    'spend_all' : 'last',
                    'spend_apg' : 'last',
                    'spend_grf' : 'last',
                    'spend_hcs': 'last',
                    'spend_tws' : 'last',
                    'spend_all_inchigh' : 'last',
                    'spend_all_inclow' : 'last',
                    'spend_all_incmiddle' : 'last',
                    'date' : 'last'
                   }
        df = df.resample('W-Mon', on = 'date').agg(agg_dict)
        df = df[df['date'] <= ('2020-08-03 00:00:00')].reset_index(drop=True) #UPDATE
        return df
    elif 'statefips' not in df.columns and 'countyfips' not in df.columns and 'cityid' not in df.columns:
        agg_dict = {'year': 'last',
                    'month': 'last',
                    'day': 'last',
                    'spend_acf': 'last',
                    'spend_aer' : 'last',
                    'spend_all' : 'last',
                    'spend_apg' : 'last',
                    'spend_grf' : 'last',
                    'spend_hcs': 'last',
                    'spend_tws' : 'last',
                    'spend_all_inchigh' : 'last',
                    'spend_all_inclow' : 'last',
                    'spend_all_incmiddle' : 'last',
                    'date' : 'last'}
        df = df.resample('W-Mon', on = 'date').agg(agg_dict)
        df = df[df['date'] <= ('2020-08-03 00:00:00')].reset_index(drop=True) #UPDATE
        return df
    if 'spend_all_inclow' in df.columns:
        df['spend_all_inclow'] = df['spend_all_inclow'].astype(float)
    else:
        print('New file converted from weekly to daily')
    

In [89]:
def womply_daily_to_weekly(df):
    '''Converts daily data to weekly data from Womply to match SafeGraph weeks'''
    if 'cityid' in df.columns and 'merchants_all' in df.columns:
        for column_name in df.columns:
            if 'merchants' in column_name:
                df[column_name] = df[column_name].astype(float)       
        agg_dict = {'year': 'last',
                    'month': 'last',
                    'day': 'last',
                    'cityid': 'last',
                    'merchants_all': 'last',
                    'merchants_ss40' : 'last',
                    #'merchants_ss60' : 'last',
                    'merchants_ss65' : 'last',
                    'merchants_ss70' : 'last',
                    'date': 'last'}
        df = df.resample('W-Mon', on = 'date').agg(agg_dict)
        df = df[df['date'] <= ('2020-08-03 00:00:00')].reset_index(drop=True) #UPDATE
        return df
    elif 'countyfips' in df.columns and 'merchants_all' in df.columns:
        agg_dict = {'year': 'last',
                    'month': 'last',
                    'day': 'last',
                    'countyfips': 'last',
                    'merchants_all': 'last',
                    'date': 'last'}
        df = df.resample('W-Mon', on = 'date').agg(agg_dict)
        df = df[df['date'] <= ('2020-08-03 00:00:00')].reset_index(drop=True) #UPDATE
        return df
    elif 'statefips' in df.columns and 'merchants_all' in df.columns:
        agg_dict = {'year' : 'last',
                    'month' : 'last',
                    'day' : 'last',
                    'statefips' : 'last',
                    'merchants_all' : 'last',
                    'merchants_inchigh' : 'last',
                    'merchants_inclow' : 'last',
                    'merchants_incmiddle' : 'last',
                    'merchants_ss40' : 'last',
                    #'merchants_ss60' : 'last',
                    'merchants_ss65' : 'last',
                    'merchants_ss70' : 'last',
                    'date' : 'last'}
        df = df.resample('W-Mon', on = 'date').agg(agg_dict)
        df = df[df['date'] <= ('2020-08-03 00:00:00')].reset_index(drop=True) #UPDATE
        return df
    elif 'statefips' not in df.columns and 'countyfips' not in df.columns and 'merchants_all' in df.columns:
        agg_dict = {'year' : 'last',
                    'month' : 'last',
                    'day' : 'last',
                    'merchants_all' : 'last',
                    'merchants_inchigh' : 'last',
                    'merchants_inclow' : 'last',
                    'merchants_incmiddle' : 'last',
                    'merchants_ss40' : 'last',
                    #'merchants_ss60' : 'last',
                    'merchants_ss65' : 'last',
                    'merchants_ss70' : 'last',
                    'date' : 'last'}
        df = df.resample('W-Mon', on = 'date').agg(agg_dict)
        df = df[df['date'] <= ('2020-08-03 00:00:00')].reset_index(drop=True) #UPDATE
        return df
    elif 'cityid' in df.columns and 'revenue_all' in df.columns:
        for column_name in df.columns:
            if 'revenue' in column_name:
                df[column_name] = df[column_name].astype(float)
        agg_dict = {'year': 'last',
                    'month': 'last',
                    'day': 'last',
                    'cityid': 'last',
                    'revenue_all': 'last',
                    'revenue_ss40' : 'last',
                    #'revenue_ss60' : 'last',
                    'revenue_ss65' : 'last',
                    'revenue_ss70' : 'last',
                    'date': 'last'}
        df = df.resample('W-Mon', on = 'date').agg(agg_dict)
        df = df[df['date'] <= ('2020-08-03 00:00:00')].reset_index(drop=True) #UPDATE
        return df
    elif 'countyfips' in df.columns and 'revenue_all' in df.columns:
        agg_dict = {'year': 'last',
                    'month': 'last',
                    'day': 'last',
                    'countyfips': 'last',
                    'revenue_all': 'last',
                    'date': 'last'}
        df = df.resample('W-Mon', on = 'date').agg(agg_dict)
        df = df[df['date'] <= ('2020-08-03 00:00:00')].reset_index(drop=True) #UPDATE
        return df
    elif 'statefips' in df.columns and 'revenue_all' in df.columns:
        agg_dict = {'year' : 'last',
                    'month' : 'last',
                    'day' : 'last',
                    'statefips' : 'last',
                    'revenue_all' : 'last',
                    'revenue_inchigh' : 'last',
                    'revenue_inclow' : 'last',
                    'revenue_incmiddle' : 'last',
                    'revenue_ss40' : 'last',
                   # 'revenue_ss60' : 'last',
                    'revenue_ss65' : 'last',
                    'revenue_ss70' : 'last',
                    'date' : 'last'
                   }
        df = df.resample('W-Mon', on = 'date').agg(agg_dict)
        df = df[df['date'] <= ('2020-08-03 00:00:00')].reset_index(drop=True) #UPDATE
        return df
    else:
        agg_dict = {'year' : 'last',
                    'month' : 'last',
                    'day' : 'last',
                    'revenue_all' : 'last',
                    'revenue_inchigh' : 'last',
                    'revenue_inclow' : 'last',
                    'revenue_incmiddle' : 'last',
                    'revenue_ss40' : 'last',
                   # 'revenue_ss60' : 'last',
                    'revenue_ss65' : 'last',
                    'revenue_ss70' : 'last',
                    'date' : 'last'
                   }
        df = df.resample('W-Mon', on = 'date').agg(agg_dict)
        df = df[df['date'] <= ('2020-08-03 00:00:00')].reset_index(drop=True) #UPDATE
        return df
    

In [90]:
def name_files(df, filename):
    '''This function takes a DataFrame and converts it into a unique csv with a unique name'''
    if 'Affinity' in filename:
        filename = filename.replace('Affinity', 'Consumer Spending')
    if 'Burning Glass' in filename:
        filename = filename.replace('Burning Glass', 'Job Postings')
    if 'UI Claims' in filename:
        filename = filename.replace('UI Claims', 'Unemployment Claims')
    if 'Womply Merchants' in filename:
        filename = filename.replace('Womply Merchants', 'Small Businesses Open')
    if 'Womply Revenue' in filename:
        filename = filename.replace('Womply Revenue', 'Small Businesses Revenue')
    if 'Zearn' in filename:
        filename = filename.replace('Zearn', 'Online Math Learning')
    if 'Daily' in filename:
        filename = filename.replace('Daily', 'Weekly')
    df.to_csv('filtered_data\{}'.format(filename), index = False)

In [120]:
def compare_rows(col1, col2):
    combined_df = pd.DataFrame({'new_col': col1, 'old_col': col2})
    combined_df['difference'] = (-0.01 <= combined_df['new_col'] - combined_df['old_col']) & (combined_df['new_col'] - combined_df['old_col'] <= 0.01) 
    print(combined_df)
    if False in list(combined_df['difference']):
        print(df)
        raise ValueError("Larger than 0.01 difference between columns")  

In [121]:
def last_week_is_same(df, filename):
    
    path = os.path.join('temp', filename)
    last_week_df = pd.read_csv(path)
    last_week_size = len(last_week_df)
    
    for i in df.columns:
        if df[i].dtype == 'int32':
            df[i] = df[i].astype('int64')
        if df[i].dtype == 'object':
            df[i] = df[i].astype('float64')
            
    for i in df.columns:
        if False in list(df.columns == last_week_df.columns):
            raise ValueError("The columns have changed")
        else:
            try:
                compare_rows(df.reset_index(drop = True).iloc[:last_week_size][i], last_week_df.iloc[:last_week_size][i])
                #assert_frame_equal(df.reset_index(drop = True).iloc[:last_week_size-1], last_week_df.iloc[:last_week_size-1])
                #df.to_csv('temp\{}'.format(filename), index = False)
                #return True
            except:
    #         df.to_csv('temp\{}'.format(filename), index = False)
    #         print('This week\'s')
    #         print(df.reset_index(drop = True).iloc[:last_week_size-1])
    #         print('Last week\'s')
    #         print(last_week_df.iloc[:last_week_size-1])
                return False
    
    return True
    

In [122]:
count = 0
for filename in os.listdir('raw_data'):
    print(os.path.join('raw_data', filename))
    count += 1 
    
    if 'City' in filename:
        sd_city(filename)
    if 'County' in filename:
        sd_county(filename)
    elif 'State' in filename:
        state_ca(filename)
    elif 'National' in filename:
        us_national(filename)
        
print(count)

raw_data\.ipynb_checkpoints
raw_data\Affinity - City - Daily.csv
    new_col  old_col  difference
0      2020     2020        True
1      2020     2020        True
2      2020     2020        True
3      2020     2020        True
4      2020     2020        True
5      2020     2020        True
6      2020     2020        True
7      2020     2020        True
8      2020     2020        True
9      2020     2020        True
10     2020     2020        True
11     2020     2020        True
12     2020     2020        True
13     2020     2020        True
14     2020     2020        True
15     2020     2020        True
16     2020     2020        True
17     2020     2020        True
18     2020     2020        True
19     2020     2020        True
20     2020     2020        True
21     2020     2020        True
22     2020     2020        True
23     2020     2020        True
24     2020     2020        True
25     2020     2020        True
26     2020     2020        True
27     2020

C:\Users\spate\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3263: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


    new_col  old_col  difference
0      2020     2020        True
1      2020     2020        True
2      2020     2020        True
3      2020     2020        True
4      2020     2020        True
5      2020     2020        True
6      2020     2020        True
7      2020     2020        True
8      2020     2020        True
9      2020     2020        True
10     2020     2020        True
11     2020     2020        True
12     2020     2020        True
13     2020     2020        True
14     2020     2020        True
15     2020     2020        True
16     2020     2020        True
17     2020     2020        True
18     2020     2020        True
19     2020     2020        True
20     2020     2020        True
21     2020     2020        True
22     2020     2020        True
23     2020     2020        True
24     2020     2020        True
25     2020     2020        True
26     2020     2020        True
27     2020     2020        True
28     2020     2020        True
29     202

    new_col  old_col  difference
0      2020     2020        True
1      2020     2020        True
2      2020     2020        True
3      2020     2020        True
4      2020     2020        True
5      2020     2020        True
6      2020     2020        True
7      2020     2020        True
8      2020     2020        True
9      2020     2020        True
10     2020     2020        True
11     2020     2020        True
12     2020     2020        True
13     2020     2020        True
14     2020     2020        True
15     2020     2020        True
16     2020     2020        True
17     2020     2020        True
18     2020     2020        True
19     2020     2020        True
20     2020     2020        True
21     2020     2020        True
22     2020     2020        True
23     2020     2020        True
24     2020     2020        True
25     2020     2020        True
26     2020     2020        True
27     2020     2020        True
28     2020     2020        True
29     202

31  -0.4100  -0.4100        True
    new_col  old_col  difference
0   -0.1350  -0.1350        True
1    0.1300   0.1300        True
2   -0.1480  -0.1480        True
3    0.1540   0.1540        True
4   -0.1470  -0.1470        True
5    0.1200   0.1200        True
6   -0.0678  -0.0678        True
7   -0.0734  -0.0734        True
8   -0.0435  -0.0435        True
9   -0.0776  -0.0776        True
10  -0.0373  -0.0373        True
11  -0.1710  -0.1710        True
12  -0.2350  -0.2350        True
13  -0.3830  -0.3830        True
14  -0.3570  -0.3570        True
15  -0.4470  -0.4470        True
16  -0.4920  -0.4920        True
17  -0.4880  -0.4880        True
18  -0.4630  -0.4630        True
19  -0.4080  -0.4080        True
20  -0.4210  -0.4210        True
21  -0.2060  -0.2060        True
22  -0.2640  -0.2640        True
23  -0.2860  -0.2860        True
24  -0.1180  -0.1180        True
25  -0.2990  -0.2990        True
26  -0.4580  -0.4580        True
27  -0.3530  -0.3530        True
28  -0.318

...

    new_col  old_col  difference
0      2020     2020        True
1      2020     2020        True
2      2020     2020        True
3      2020     2020        True
4      2020     2020        True
5      2020     2020        True
6      2020     2020        True
7      2020     2020        True
8      2020     2020        True
9      2020     2020        True
10     2020     2020        True
11     2020     2020        True
12     2020     2020        True
13     2020     2020        True
14     2020     2020        True
15     2020     2020        True
16     2020     2020        True
17     2020     2020        True
18     2020     2020        True
19     2020     2020        True
20     2020     2020        True
21     2020     2020        True
22     2020     2020        True
23     2020     2020        True
24     2020     2020        True
25     2020     2020        True
26     2020     2020        True
27     2020     2020        True
28     2020     2020        True
29     202

raw_data\Womply Revenue - County - Daily.csv
    new_col  old_col  difference
0      2020     2020        True
1      2020     2020        True
2      2020     2020        True
3      2020     2020        True
4      2020     2020        True
5      2020     2020        True
6      2020     2020        True
7      2020     2020        True
8      2020     2020        True
9      2020     2020        True
10     2020     2020        True
11     2020     2020        True
12     2020     2020        True
13     2020     2020        True
14     2020     2020        True
15     2020     2020        True
16     2020     2020        True
17     2020     2020        True
18     2020     2020        True
19     2020     2020        True
20     2020     2020        True
21     2020     2020        True
22     2020     2020        True
23     2020     2020        True
24     2020     2020        True
25     2020     2020        True
26     2020     2020        True
27     2020     2020        Tru

raw_data\Womply Revenue - State - Daily.csv
    new_col  old_col  difference
0      2020     2020        True
1      2020     2020        True
2      2020     2020        True
3      2020     2020        True
4      2020     2020        True
5      2020     2020        True
6      2020     2020        True
7      2020     2020        True
8      2020     2020        True
9      2020     2020        True
10     2020     2020        True
11     2020     2020        True
12     2020     2020        True
13     2020     2020        True
14     2020     2020        True
15     2020     2020        True
16     2020     2020        True
17     2020     2020        True
18     2020     2020        True
19     2020     2020        True
20     2020     2020        True
21     2020     2020        True
22     2020     2020        True
23     2020     2020        True
24     2020     2020        True
25     2020     2020        True
26     2020     2020        True
27     2020     2020        True

    new_col  old_col  difference
0      2020     2020        True
1      2020     2020        True
2      2020     2020        True
3      2020     2020        True
4      2020     2020        True
5      2020     2020        True
6      2020     2020        True
7      2020     2020        True
8      2020     2020        True
9      2020     2020        True
10     2020     2020        True
11     2020     2020        True
12     2020     2020        True
13     2020     2020        True
14     2020     2020        True
15     2020     2020        True
16     2020     2020        True
    new_col  old_col  difference
0         1        1        True
1         1        1        True
2         1        1        True
3         2        2        True
4         2        2        True
5         2        2        True
6         2        2        True
7         3        3        True
8         3        3        True
9         3        3        True
10        3        3        True
11        

16        6        6        True
    new_col  old_col  difference
0   0.01900  0.01900        True
1   0.07050  0.07050        True
2   0.01700  0.01700        True
3   0.11000  0.11000        True
4   0.12300  0.12300        True
5   0.03380  0.03380        True
6  -0.03440 -0.03440        True
7   0.12900  0.12900        True
8   0.11300  0.11300        True
9   0.14600  0.14600        True
10 -0.32900 -0.32900        True
11 -0.34700 -0.34700        True
12 -0.26900 -0.26900        True
13 -0.24200 -0.24200        True
14 -0.08990 -0.08990        True
15 -0.00435 -0.00435        True
16 -0.05020 -0.05020        True
    new_col  old_col  difference
0    0.0391   0.0391        True
1    0.0477   0.0477        True
2   -0.0742  -0.0742        True
3    0.1600   0.1600        True
4    0.2230   0.2230        True
5    0.0277   0.0277        True
6   -0.0538  -0.0538        True
7    0.2420   0.2420        True
8    0.2540   0.2540        True
9    0.2890   0.2890        True
10  -0.063

#### -- Everything under this cell is for visualizing the data --

In [3]:
#Affinity  

# county
path_1 = os.path.join('raw_data', 'Affinity - City - Daily.csv')
affinity_county = pd.read_csv(path_1, low_memory = False)
affinity_county = affinity_county[affinity_county['cityid'] == 6]
affinity_county.tail(16)

# state
# path_2 = os.path.join('raw_data', 'Affinity - State - Weekly.csv')
# affinity_state = pd.read_csv(path_2)
# affinity_state[affinity_state['statefips'] == 6]['spend_all_inchigh'].astype(float)

# # national 
# path_3 = os.path.join('raw_data', 'Affinity - National - Weekly.csv')
# affinity_national = pd.read_csv(path_3)
# affinity_national

,year,month,day,cityid,spend_acf,spend_aer,spend_all,spend_apg,spend_grf,spend_hcs,spend_tws
10055,2020,7,25,6,-.431,-.645,-.143,-.205,.13,-.179,-.567
10105,2020,7,26,6,-.431,-.634,-.145,-.199,.123,-.194,-.566
10155,2020,7,27,6,-.425,-.636,-.146,-.201,.129,-.191,-.565
10205,2020,7,28,6,-.419,-.638,-.146,-.203,.134,-.187,-.564
10255,2020,7,29,6,-.414,-.64,-.147,-.205,.14,-.184,-.563
10305,2020,7,30,6,-.408,-.643,-.147,-.207,.146,-.18,-.562
10355,2020,7,31,6,-.402,-.645,-.148,-.208,.152,-.177,-.561
10405,2020,8,1,6,-.396,-.647,-.148,-.21,.157,-.173,-.56
10455,2020,8,2,6,-.391,-.649,-.149,-.212,.163,-.17,-.559
10505,2020,8,3,6,-.389,-.638,-.145,-.211,.164,-.165,-.552


In [9]:
path_1 = os.path.join('temp', 'Affinity - State - Daily.csv')
affinity_county = pd.read_csv(path_1, low_memory = False)
affinity_county.columns

Index(['year', 'month', 'day', 'statefips', 'spend_acf', 'spend_aer',
       'spend_all', 'spend_apg', 'spend_grf', 'spend_hcs', 'spend_tws',
       'spend_all_inchigh', 'spend_all_inclow', 'spend_all_incmiddle'],
      dtype='object')

In [11]:
affinity_state[affinity_state['statefips'] == 6]['spend_all_inclow']

4        -.0104
55        .0195
106     .000637
157     -.00947
208       .0504
259      .00883
310       .0105
361     -.00599
412      .00659
463       -.047
514       -.183
565       -.345
616       -.318
667       -.381
718        -.34
769       -.295
820       -.253
871       -.242
922       -.241
973        -.23
1024       -.24
1075      -.187
1126      -.196
1177      -.123
1228       -.16
1279      -.112
1330      -.145
Name: spend_all_inclow, dtype: object

In [303]:
# Burning Glass

# state
path2_2 = os.path.join('raw_data', 'Burning Glass - State - Weekly.csv')
b_glass_state = pd.read_csv(path2_2)
b_glass_state = b_glass_state[b_glass_state['statefips'] == 6]
b_glass_state = b_glass_state.rename(columns={"day_endofweek": "day"})
b_glass_state['date'] = pd.to_datetime(b_glass_state[['year','month','day']])
b_glass_state = b_glass_state.sort_values(by=['date'])
b_glass_state = b_glass_state.rename(columns={"day": "day_endofweek"})
b_glass_state = b_glass_state.drop(['date'], axis = 1)
b_glass_state.head()

# # national 
# path2_3 = os.path.join('raw_data', 'Burning Glass - National - Weekly.csv')
# b_glass_national = pd.read_csv(path2_3)
# b_glass_national.head()

,year,month,day_endofweek,statefips,bg_posts,bg_posts_ss30,bg_posts_ss55,bg_posts_ss60,bg_posts_ss65,bg_posts_ss70,bg_posts_jz1,bg_posts_jz2,bg_posts_jz3,bg_posts_jz4,bg_posts_jz5,bg_posts_jzgrp12,bg_posts_jzgrp345
136,2020,1,10,6,-0.0934,-0.1590,-0.0742,-0.205,-0.0998,-0.1210,-0.0679,-0.0790,-0.0829,-0.1240,-0.0663,-0.0780,-0.102
126,2020,1,17,6,0.1470,0.1140,0.0816,0.197,0.2260,0.3350,0.3990,0.1840,0.1800,0.0741,0.2000,0.2010,0.128
121,2020,1,24,6,-0.1840,-0.0628,-0.1810,-0.252,-0.2620,-0.2350,-0.2830,-0.1800,-0.1860,-0.1670,-0.2630,-0.1890,-0.187
132,2020,1,31,6,0.1300,0.1080,0.1740,0.260,0.1360,0.0208,-0.0473,0.0755,0.0892,0.2160,0.1290,0.0654,0.161
115,2020,2,7,6,-0.1150,-0.1540,-0.0475,-0.175,-0.0954,-0.0894,-0.1110,-0.1010,-0.1120,-0.1370,-0.1010,-0.1010,-0.123


In [12]:
# UI Claims

# county
# path7_1  = os.path.join('raw_data', 'UI Claims - City - Weekly.csv')
# ui = pd.read_csv(path7_1)
# ui[(ui['cityid'] == 6)].tail() #& (ui['month'] == 2)].head()

# # state
path7_2  = os.path.join('raw_data', 'UI Claims - State - Weekly.csv')
ui = pd.read_csv(path7_2)
ui[(ui['statefips'] == 6)].tail()

# # national
# path7_3  = os.path.join('raw_data', 'UI Claims - National - Weekly.csv')
# ui = pd.read_csv(path7_3)
# ui.head()

,year,month,day_endofweek,statefips,initial_claims,initial_claims_rate,total_claims,total_claims_rate
1381,2020,7,11,6,284914,1.47,3231721,16.6
1432,2020,7,18,6,289594,1.49,2994498,15.4
1483,2020,7,25,6,244653,1.26,3385272,17.4
1534,2020,8,1,6,222043,1.14,3011525,15.5
1585,2020,8,8,6,202509,1.04,3163697,16.3


In [29]:
import numpy as np

In [50]:
# Womply Merchants

# county
path8_1  = os.path.join('raw_data', 'Womply Merchants - City - Daily.csv')
womply_merchants = pd.read_csv(path8_1)
womply_merchants = womply_merchants[(womply_merchants['cityid'] == 6)]
womply_merchants = womply_merchants.rename(columns={"day_endofweek": "day"})
womply_merchants['date'] = pd.to_datetime(womply_merchants[['year','month','day']])
womply_merchants = womply_merchants.sort_values(by=['date'])
womply_merchants.head(11)[['date','merchants_ss60']]

,date,merchants_ss60
5,2020-01-10,.0276
57,2020-01-11,.0319
109,2020-01-12,.0311
161,2020-01-13,.0342
213,2020-01-14,.0326
265,2020-01-15,.0321
317,2020-01-16,.018
369,2020-01-17,.0154
421,2020-01-18,.00807
473,2020-01-19,.0063


In [51]:
agg_dict = {'year': 'last',
          'month': 'last',
          'day': 'last',
          'countyfips': 'last',
          'merchants_all': 'last',
          'date': 'last'}

# # resampled dataframe
# # 'W' means weekly aggregation
# r_df = df.resample('W').agg(agg_dict)

womply_merchants = womply_merchants.resample('W-Mon', on = 'date').agg(agg_dict)
womply_merchants = womply_merchants[womply_merchants['date'] <= ('2020-07-13 00:00:00')].reset_index(drop=True)
len(womply_merchants)

SpecificationError: nested renamer is not supported

In [53]:
# Womply Revenue

# county
path9_1  = os.path.join('raw_data', 'Womply Merchants - County - Daily.csv')
womply_revenue = pd.read_csv(path9_1)
womply_revenue[(womply_revenue['countyfips'] == 6073) & (womply_revenue['month'] == 2)].head()

,year,month,day,countyfips,merchants_all
46739,2020,2,1,6073,-0.00605
48857,2020,2,2,6073,-0.00568
50974,2020,2,3,6073,-0.00144
53092,2020,2,4,6073,0.00119
55210,2020,2,5,6073,0.00232


In [126]:
# Zearn

path10_1  = os.path.join('raw_data', 'Zearn - County - Weekly.csv')
zearn = pd.read_csv(path10_1)
zearn = zearn[(zearn['countyfips'] == 6073)]
zearn = zearn.rename(columns={"day_endofweek": "day"})
zearn['date'] = pd.to_datetime(zearn[['year','month','day']])
zearn = zearn.sort_values(by=['date'])
zearn = zearn.rename(columns={"day": "day_endofweek"})
zearn = zearn.drop(['date'], axis = 1)
zearn.head()



,year,month,day_endofweek,countyfips,engagement,badges,imputed_from_cz
2535,2020,1,12,6073,-.0322,-.0792,0
2541,2020,1,19,6073,.0148,.0368,0
2542,2020,1,26,6073,-.0295,-.113,0
2549,2020,2,2,6073,.154,.191,0
2539,2020,2,9,6073,.163,.3,0


In [46]:
d = {'col1': [1, 2,3,4], 'col2': [9,10,11,12], 'col3': [5,6,7,8]}
df = pd.DataFrame(data=d)
df

,col1,col2,col3
0,1,9,5
1,2,10,6
2,3,11,7
3,4,12,8


In [66]:
d2 = {'col1': [1, 2,3,3], 'col2': [9,10,11,12], 'col3': [5,6,7,8]}
df2 = pd.DataFrame(data=d2)
df2

,col1,col2,col3
0,1,9,5
1,2,10,6
2,3,11,7
3,3,12,8


In [67]:
print(df.loc[~(df['col1'] == d2['col1'])])
print(df2.loc[~(df2['col1'] == d['col1'])])


   col1  col2  col3
3     4    12     8
   col1  col2  col3
3     3    12     8


In [68]:
if False in list(df.columns == df2.columns):
    print(False)
else:
    print(True)

True


In [69]:
list(df.columns == df2.columns)

[True, True, True]

In [113]:
df3 = pd.DataFrame({'col1': df.iloc[:3]['col2'], 'col2': [10, 9, 10]})
df3['h'] = (df3['col1'] - df3['col2'] < 1) & (df3['col1'] - df3['col2'] > -1)
df3

,col1,col2,h
0,9,10,False
1,10,9,False
2,11,10,False


In [79]:
 list(df.iloc[:3]['col2'])

[9, 10, 11]

In [123]:
'C:/Users/spate/Downloads/Track the Recovery/temp/{}'.format('hi')

'C:/Users/spate/Downloads/Track the Recovery/temp/hi'